# Simulation template

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 707 ms, sys: 267 ms, total: 973 ms
Wall time: 565 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = SSH1_PERIODIC_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.72 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy", "n_estimators": 25, "n_jobs": -1}
allowed_windings = [-1, 0, 1, 2]
val_split = 0.5
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh2/periodic_220_6561.csv"
model_name = "RandomForestClassifier"
simulation_dir = "/home/rio/ssh_simulations/ssh2/periodic_220_6561"
random_state = 401


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

/home/rio/ml_top_phases/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1443420 entries, 0 to 1443419
Columns: 228 entries, id to feat219
dtypes: float64(223), int32(3), object(2)
memory usage: 2.4+ GB
CPU times: user 25.5 s, sys: 1.26 s, total: 26.7 s
Wall time: 26.7 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.130846,-0.300844,...,0.080274,0.115552,0.174807,-0.244872,-0.319478,0.059064,0.000568,0.244891,0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244891,-0.300844,-0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.139243,-0.040832,...,0.177651,-0.085976,-0.069808,-0.109102,-0.140759,0.184935,0.184071,-0.052390,-0.040832,-0.139243
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.048483,0.015611,...,-0.065026,0.044350,0.035714,0.029502,0.046097,-0.070102,-0.070512,0.025241,0.015611,0.048483
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.005059,0.009352,...,-0.013734,0.009909,0.002329,0.008250,0.011761,-0.009332,-0.002720,-0.004584,-0.009352,0.005059


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  220
n_hamiltonians:  6561
n_ts:  2
CPU times: user 141 µs, sys: 8 µs, total: 149 µs
Wall time: 94.9 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.5450388660265203
% val:  0.0
% test:  0.45496113397347965
% train + val + test:  1.0


number of train hamiltonians:  3576
number of val hamiltonians:  0
number of test hamiltonians:  2985
total number of hamiltonians:  6561


train ids:  [41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [01:10<1:55:35, 70.06s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [02:20<1:54:45, 70.26s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [1:55:59<00:00, 69.00s/it]

running experiments: 100%|██████████| 100/100 [1:55:59<00:00, 69.59s/it]

CPU times: user 12h 47min 12s, sys: 26.3 s, total: 12h 47min 38s
Wall time: 1h 55min 59s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.130846,-0.300844,...,0.080274,0.115552,0.174807,-0.244872,-0.319478,0.059064,0.000568,0.244891,0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244891,-0.300844,-0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.139243,-0.040832,...,0.177651,-0.085976,-0.069808,-0.109102,-0.140759,0.184935,0.184071,-0.052390,-0.040832,-0.139243
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.048483,0.015611,...,-0.065026,0.044350,0.035714,0.029502,0.046097,-0.070102,-0.070512,0.025241,0.015611,0.048483
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.005059,0.009352,...,-0.013734,0.009909,0.002329,0.008250,0.011761,-0.009332,-0.002720,-0.004584,-0.009352,0.005059
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.050074,-0.011117,...,0.049582,0.005255,0.001717,-0.054642,-0.055678,0.050847,0.051142,-0.002840,-0.011117,-0.050074
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.009938,-0.026818,...,-0.013334,0.022656,0.023129,-0.013172,-0.023569,-0.006879,-0.004162,0.025408,0.026818,-0.009938
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.014007,0.025469,...,-0.039242,0.030036,0.008341,0.021972,0.032295,-0.027020,-0.008046,-0.011692,-0.025469,0.014007
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.053983,-0.019696,...,0.075379,-0.071666,-0.058670,-0.017526,-0.045907,0.086976,0.089522,-0.040115,-0.019696,-0.053983
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.068700,0.013277,...,-0.056877,-0.023603,-0.014327,0.078240,0.077328,-0.060835,-0.062904,-0.003322,0.013277,0.068700


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 250 ms, sys: 0 ns, total: 250 ms
Wall time: 249 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
1423385,6469,H_1.95_1.5_-1.00952.dat,1.95,1.50,-1.009522,999,1,test,0.070813,0.044133,...,0.077175,0.134385,0.068929,0.008387,-0.090104,-0.026178,-0.069492,-0.091007,-0.044133,-0.070813
144562,657,H_-1.6_-1.55_-1.12856.dat,-1.60,-1.55,-1.128564,999,1,test,0.027254,-0.004244,...,-0.031213,-0.002108,0.001853,-0.029327,0.029453,0.030186,-0.029396,-0.002288,0.004244,-0.027254
1178767,5358,H_1.3_-1.4_1.001.dat,1.30,-1.40,1.001004,1,1,train,0.001807,-0.001602,...,0.005426,0.012749,0.011228,0.013351,0.007598,0.003859,0.000886,-0.001606,0.001602,-0.001807
1381383,6279,H_1.85_0.1_-1.00065.dat,1.85,0.10,-1.000651,999,1,test,-0.087449,0.087504,...,-0.071500,0.072476,-0.077944,0.079830,-0.085410,0.083820,-0.085946,0.086752,-0.087504,0.087449
292213,1328,H_-1.2_-0.4_-0.00406736.dat,-1.20,-0.40,-0.004067,0,0,train,0.063142,-0.000718,...,0.056974,-0.032928,0.083713,0.084549,-0.034459,0.056768,-0.095094,-0.066186,0.000718,-0.063142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152796,694,H_-1.6_0.3_-0.00114167.dat,-1.60,0.30,-0.001142,0,0,val,-0.069166,0.100509,...,0.015050,-0.102759,-0.048322,-0.048418,-0.061269,-0.039565,-0.106792,0.013076,-0.100509,0.069166
458985,2086,H_-0.75_1.05_0.000210896.dat,-0.75,1.05,0.000211,0,0,val,-0.033899,-0.083698,...,-0.012393,0.017196,0.016539,-0.089587,0.047578,0.057826,-0.038623,0.064341,-0.083698,-0.033899
586782,2667,H_-0.4_1.75_2.00106.dat,-0.40,1.75,2.001059,999,0,test,-0.048566,-0.031077,...,0.047501,0.025381,0.073769,-0.023820,0.085762,-0.072436,0.019131,-0.075955,-0.031077,-0.048566
199522,906,H_-1.45_-1.25_-1.02688.dat,-1.45,-1.25,-1.026878,999,0,test,-0.055711,0.075612,...,-0.029700,-0.005805,0.007635,0.016531,-0.035761,0.085363,-0.022606,0.085259,-0.075612,0.055711


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.27251943301326015
% val:  0.27251943301326015
% test:  0.45496113397347965
% train + val + test:  1.0


number of train hamiltonians:  1788
number of val hamiltonians:  1788
number of test hamiltonians:  2985
total number of hamiltonians:  6561


train ids:  [2534, 3740, 3666, 2302, 3590, 3916, 301, 4554, 4701, 2615, 122, 72, 207, 1415, 706, 5846, 2522, 2733, 4418, 868, 3090, 5841, 2015, 4143, 2761, 2067, 2604, 2857, 2779, 2241, 3565, 6485, 941, 2852, 3810, 1259, 1488, 1113, 3347, 4632, 2611, 4071, 445, 128, 6404, 4490, 4312, 1734, 1163, 5359, 5268, 3352, 4005, 4212, 2612, 2526, 4706, 3777, 290, 2800, 1513, 4819, 927, 2171, 1596, 2174, 791, 1365, 1016, 4562, 6162, 5854, 3180, 1207, 2961, 3106, 287, 6324, 6158, 3580, 4329, 1740, 1103, 3514, 6403, 4811, 3615, 5751, 5124, 1412, 4787, 5441, 2163, 3997, 1133, 4482, 1665, 3696, 3688, 6414, 527, 5606, 863, 5601, 1499, 1091, 943, 485, 1757, 1189, 3275, 5767, 3767, 1619, 5925, 1162, 3339, 3119, 1040, 151, 1493, 4970, 1295, 5278, 639

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 116 µs, sys: 0 ns, total: 116 µs
Wall time: 83 µs


,t1,t2,type_of,0,1,phase,pred_phase
id,,,,,,,
0,-2.0,-2.00,test,0.613636,0.386364,999,-1
1,-2.0,-1.95,test,0.631818,0.368182,999,-1
2,-2.0,-1.90,test,0.595455,0.404545,999,-1
3,-2.0,-1.85,test,0.595455,0.404545,999,-1
4,-2.0,-1.80,test,0.563636,0.436364,999,-1
...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.668182,0.331818,999,-1
6557,2.0,1.85,test,0.636364,0.363636,999,-1
6558,2.0,1.90,test,0.722727,0.277273,999,-1


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  1443420


,id,phase,pred_phase,type_of
0,0,999,0,test
1,0,999,0,test
2,0,999,1,test
3,0,999,1,test
4,0,999,1,test
...,...,...,...,...
1443415,6560,999,1,test
1443416,6560,999,0,test
1443417,6560,999,1,test
1443418,6560,999,0,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9998779743746187,
 'eigenvector_val': 0.9897015456579215,
 'eigenvector_test': 0.7961898395721925,
 'hamiltonian_train': 0.0,
 'hamiltonian_val': 0.0,
 'hamiltonian_test': 0.00980392156862745}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}